**Goal:**

- The objective of this project is to predict whether loan can be granted to a person or not.

In [0]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score

**DATASET PROVIDED:**

Two dataset has been provided for this problem.

**loan_table** consists of loan details.

**borrower_table** consists of borrower details.

There is a total of **101100 records** in both the tables, and they can be linked through the column **loan_id**.


In [0]:
loan_data=pd.read_csv('loan_table.csv')
borrower_data=pd.read_csv('borrower_table.csv')

In [4]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101100 entries, 0 to 101099
Data columns (total 5 columns):
loan_id         101100 non-null int64
loan_purpose    101100 non-null object
date            101100 non-null object
loan_granted    101100 non-null int64
loan_repaid     47654 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 3.9+ MB


In [5]:
loan_data.head()

,loan_id,loan_purpose,date,loan_granted,loan_repaid
0,19454,investment,2012-03-15,0,NaN
1,496811,investment,2012-01-17,0,NaN
2,929493,other,2012-02-09,0,NaN
3,580653,other,2012-06-27,1,1.0
4,172419,business,2012-05-21,1,0.0


In [6]:
borrower_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101100 entries, 0 to 101099
Data columns (total 12 columns):
loan_id                                            101100 non-null int64
is_first_loan                                      101100 non-null int64
fully_repaid_previous_loans                        46153 non-null float64
currently_repaying_other_loans                     46153 non-null float64
total_credit_card_limit                            101100 non-null int64
avg_percentage_credit_card_limit_used_last_year    94128 non-null float64
saving_amount                                      101100 non-null int64
checking_amount                                    101100 non-null int64
is_employed                                        101100 non-null int64
yearly_salary                                      101100 non-null int64
age                                                101100 non-null int64
dependent_number                                   101100 non-null int64
dtypes: fl

In [7]:
borrower_data.head()

,loan_id,is_first_loan,fully_repaid_previous_loans,currently_repaying_other_loans,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,is_employed,yearly_salary,age,dependent_number
0,289774,1,NaN,NaN,8000,0.49,3285,1073,0,0,47,3
1,482590,0,1.0,0.0,4500,1.03,636,5299,1,13500,33,1
2,135565,1,NaN,NaN,6900,0.82,2085,3422,1,24500,38,8
3,207797,0,1.0,0.0,1200,0.82,358,3388,0,0,24,1
4,828078,0,0.0,0.0,6900,0.80,2138,4282,1,18100,36,1


Linking both dataset through the common column **'loan_id'**

In [0]:
loan_data_full=borrower_data.merge(loan_data,on='loan_id')

In [9]:
loan_data_full.head()

,loan_id,is_first_loan,fully_repaid_previous_loans,currently_repaying_other_loans,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,is_employed,yearly_salary,age,dependent_number,loan_purpose,date,loan_granted,loan_repaid
0,289774,1,NaN,NaN,8000,0.49,3285,1073,0,0,47,3,business,2012-01-31,0,NaN
1,482590,0,1.0,0.0,4500,1.03,636,5299,1,13500,33,1,investment,2012-11-02,0,NaN
2,135565,1,NaN,NaN,6900,0.82,2085,3422,1,24500,38,8,other,2012-07-16,1,1.0
3,207797,0,1.0,0.0,1200,0.82,358,3388,0,0,24,1,investment,2012-06-05,0,NaN
4,828078,0,0.0,0.0,6900,0.80,2138,4282,1,18100,36,1,emergency_funds,2012-11-28,0,NaN


**OBSERVATIONS:**



- Four columns has null values in the dataset.

In [10]:
loan_data_full.isna().sum()

loan_id                                                0
is_first_loan                                          0
fully_repaid_previous_loans                        54947
currently_repaying_other_loans                     54947
total_credit_card_limit                                0
avg_percentage_credit_card_limit_used_last_year     6972
saving_amount                                          0
checking_amount                                        0
is_employed                                            0
yearly_salary                                          0
age                                                    0
dependent_number                                       0
loan_purpose                                           0
date                                                   0
loan_granted                                           0
loan_repaid                                        53446
dtype: int64

OBSERVATION 1:

**loan_granted column** describes whether loan has been granted or not with 0 for loan not granted and 1 for loan granted.

**loan_repaid column** describes whether loan(if granted) has been repaid or not.It has 0 to represent loan has not been repaid, 1 to represent loan has been repaid and NaN to represent empty value.

**NOTE:** If loan is **not granted**  for a person (record), then for that person (record) loan repaid will be **empty**, because there will be nothing to repay if the loan has never been taken from the beginning. Hence, **loan_repaid** column has **NaN** values.

In [11]:
print(loan_data_full['loan_granted'].value_counts())

0    53446
1    47654
Name: loan_granted, dtype: int64


In [12]:
print(loan_data_full['loan_repaid'].isna().sum())

53446


It can be noted from the above two code snippets that the number of records with loan not granted is 53446, and the number of records with null values for loan_repaid is 53446.

OBSERVATION 2:

**is_first_loan column**  describes whether the person is taking the loan for the first time or not, with 0 for **no** and 1 for **yes**.

**fully_repaid_previous_loans column** represents if the person has repaid his previous loans or not with 1 for yes and 0 for no and NaN for empty value.

**currently_repaying_other_loans column** represents if the person is currently repaying any other loans or not if 1 for yes and 0 for no and NaN for empty value.

**NOTE:** If **is_taking_loan** (for a record) is 1, then that record will definitely have NaN value for the columns **fully_repaid_previous_loans** and **currently_repaying_other_loans** column, because that person is taking a loan for the first time, and he/she cannot have previous/current loans. Hence, these two columns have **NaN** values.  



In [13]:
print(loan_data_full['is_first_loan'].value_counts())

1    54947
0    46153
Name: is_first_loan, dtype: int64


In [14]:
print(loan_data_full['fully_repaid_previous_loans'].isna().sum())

54947


In [15]:
print(loan_data_full['currently_repaying_other_loans'].isna().sum())

54947


It can be noted from the above three code snippets that the number of records having 1 for is_first_loan is 54947, and the number of records with null values for both the columns fully_repaid_previous_loans and currently_repaying_other_loans is also 54947. 

OBSERVATION 3:

**total_credit_card_limit column** represents the credit card limit for a person. It consists of numerical data.

**avg_percentage_credit_card_limit_used_last_year column** represents how much a person used from his/her credit card limit on a average in the past 12 months in terms of percentage. It is ideally between 0 and 1, and it may go beyond 1 of a person used above his/her credit card limit.

**NOTE:** For all those records that have **total_credit_card_limit** as **0**, the **avg_percentage_credit_card_limit_used_last_year** for that record will be **NaN**, as that person cannot use a credit card, and hence there will be no history for it.

In [16]:
loan_data_full['total_credit_card_limit'].value_counts()

0        6972
3300     1981
4000     1918
3500     1914
4300     1896
         ... 
13300       1
13200       1
13500       1
12700       1
12500       1
Name: total_credit_card_limit, Length: 125, dtype: int64

In [17]:
loan_data_full['avg_percentage_credit_card_limit_used_last_year'].isna().sum()

6972

It can be noted from above two code snippets that there is 6972 records with total credit card limit as 0, and the number of null values in avg_percentage_credit_card_limit_used_last_year is also 6972.

**DATA PRE-PROCESSING:**

Merging the columns laon_granted and loan_repaid and representing it with a new column **loan_status** having values 

0 - Loan not granted

1 - Loan granted and repaid

2 - Loan granted and not repaid

In the above three cases, **1** is the most preferred case.

In [0]:
def status(row):
  if row['loan_granted']==0:
    return 0
  if row['loan_granted'] == 1 and row['loan_repaid'] == 1:
    return 1
  if row['loan_granted'] == 1 and row['loan_repaid'] == 0:
    return 2

In [0]:
loan_data_full['loan_status']=loan_data_full.apply(status,axis=1)

Remove unwanted and redundant data.

In [0]:
loan_data_full=loan_data_full.drop(['loan_repaid', 'loan_granted','loan_id', 'date'], axis=1)

I have filled all the records having NaN values for the column **avg_percentage_credit_card_limit_used_last_year** with 0 , as the total credit card limit value for those records is 0. 

In [0]:
loan_data_full['avg_percentage_credit_card_limit_used_last_year']=loan_data_full['avg_percentage_credit_card_limit_used_last_year'].fillna(0)


One hot encoded the columns fully_repaid_previous_loans, currently_repaying_other_loans, is_employed, dependent_number, loan_purpose using pandas function pd.get_dummies()

**NOTE:** Once the above mentioned columns have been encoded, the null values in the columns **fully_repaid_previous_loans** and **currently_repaying_other_loans** will be eliminated automatically.

In [0]:
one_hot_encoded=pd.get_dummies(loan_data_full,columns=['fully_repaid_previous_loans','currently_repaying_other_loans','is_employed','dependent_number','loan_purpose'])

In [23]:
one_hot_encoded.head()

,is_first_loan,total_credit_card_limit,avg_percentage_credit_card_limit_used_last_year,saving_amount,checking_amount,yearly_salary,age,loan_status,fully_repaid_previous_loans_0.0,fully_repaid_previous_loans_1.0,currently_repaying_other_loans_0.0,currently_repaying_other_loans_1.0,is_employed_0,is_employed_1,dependent_number_0,dependent_number_1,dependent_number_2,dependent_number_3,dependent_number_4,dependent_number_5,dependent_number_6,dependent_number_7,dependent_number_8,loan_purpose_business,loan_purpose_emergency_funds,loan_purpose_home,loan_purpose_investment,loan_purpose_other
0,1,8000,0.49,3285,1073,0,47,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,0,4500,1.03,636,5299,13500,33,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,1,6900,0.82,2085,3422,24500,38,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0,1200,0.82,358,3388,0,24,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,0,6900,0.80,2138,4282,18100,36,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0


There is no NaN values in the one hot encoded dataset.

In [24]:
one_hot_encoded.isna().any().sum()

0

Split the dataset in to dependent and independent variables.

In [0]:
y=one_hot_encoded['loan_status']
X=one_hot_encoded.drop(columns='loan_status',axis=1)

Split the dataset in to train and test dataset

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Performance metrics

In [0]:
def performance_metrics(y_test,pred):
  print("Performance metrics:")
  print("Accuracy  :",accuracy_score(y_test,pred))
  print("Precision :",precision_score(y_test,pred,average='weighted'))
  print("Recall    :",recall_score(y_test,pred,average='weighted'))
  print("f1 score  :",f1_score(y_test, pred, average='weighted'))

Create a random forest classifier to train the data.

In [28]:
model = RandomForestClassifier(max_depth=20)
model.fit(X_train, y_train)
pred=model.predict(X_test)

performance_metrics(y_test,pred)
print("\nNOTE: Scores have been calculated without scaling the dataset,as Random forest\n      is a tree based model and does not require feature scaling")

Performance metrics:
Accuracy  : 0.7468348170128586
Precision : 0.7663183665660072
Recall    : 0.7468348170128586
f1 score  : 0.7477448719179032

NOTE: Scores have been calculated without scaling the dataset,as Random forest
      is a tree based model and does not require feature scaling




---



**EXPERIMENTING WITH OTHER MODELS:**

In [29]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(X_train,y_train)
pred=model_nb.predict(X_test)

performance_metrics(y_test,pred)

Performance metrics:
Accuracy  : 0.6905539070227498
Precision : 0.7223434519718748
Recall    : 0.6905539070227498
f1 score  : 0.6907932699132106


In [30]:
model = RandomForestClassifier(max_depth=40)
model_nb = GaussianNB()

model_vc = VotingClassifier(estimators=[
       ('rf', model), ('gnb', model_nb)],
       voting='soft', weights=[2,1],
       flatten_transform=True)
model_vc = model_vc.fit(X_train, y_train)
pred=model_vc.predict(X_test)

performance_metrics(y_test,pred)

Performance metrics:
Accuracy  : 0.7442136498516321
Precision : 0.7626691256606534
Recall    : 0.7442136498516321
f1 score  : 0.744930446101379
